
# **Constructing Missing Parts In An Image Using Auto Encoders**


[<img src="https://github.com/shubham0204/Privacy_Policy_Texts/blob/master/notebook_button.png?raw=true" width="210" height="40" align="center">](https://medium.com/@equipintelligence/neural-implanting-with-autoencoders-and-tensorflow-9c2c7b532198)

---

In this notebook, we'll learn to make a model which can construct missing parts of an image. Given an image ( which has some part of it missing ), our model will try to construct that missing part with the help of AutoEncoders.




## 1) Downloading the images

First, we'll download some images of mountains which I've hosted on GitHub. This images are from the [Intel Image Classification](https://www.kaggle.com/puneet6060/intel-image-classification) dataset by [Puneet Bansal](https://www.kaggle.com/puneet6060) on [Kaggle](https://www.kaggle.com/).


In [ ]:

import tensorflow as tf
import matplotlib.pyplot as plt
import os
import numpy as np
import random
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow.keras.layers import Conv2D , Conv2DTranspose , MaxPooling2D , UpSampling2D , Input , LeakyReLU , Concatenate

!wget https://github.com/shubham0204/Dataset_Archives/blob/master/mountain_images.zip?raw=true -O images.zip
!unzip images.zip



## 2) Preparing the input data

Our job is to construct the missing parts in an image. So, as inputs to our model, we have to randomly take a square part ( whose side length is `window_size` ) from an image and replace it with a zero array of the same shape.


In [ ]:

x = []
y = []
input_size = ( 228 , 228 , 3 )

# Take out a square region of side 50 px.
window_size = 50

# Store the original images as target images.
for name in os.listdir( 'mountain_images/' ):
    image = Image.open( 'mountain_images/{}'.format( name ) ).resize( input_size[0:2] )
    image = np.asarray( image ).astype( np.uint8 )
    y.append( image )

for name in os.listdir( 'mountain_images/' ):
    image = Image.open( 'mountain_images/{}'.format( name ) ).resize( input_size[0:2] )
    image = np.asarray( image ).astype( np.uint8 )
    # Generate random X and Y coordinates within the image bounds.
    px , py = random.randint( 0 , input_size[0] - window_size ) , random.randint( 0 , input_size[0] - window_size )
    # Take that part of the image and replace it with a zero array. This makes the "missing" part of the image.
    image[ px : px + window_size , py : py + window_size , 0:3 ] = np.zeros( ( window_size , window_size , 3 ) )
    # Append it to an array
    x.append( image )
    
#  Normalize the images
x = np.array( x ) / 255
y = np.array( y ) / 255

print( x.shape )
print( y.shape )

# Train test split
x_train, x_test, y_train, y_test = train_test_split( x , y , test_size=0.2 )

# Plot an image to see what happened!
plt.imshow( x_test[0] )
plt.show()



## 3) The Auto Encoder Model

We'll create a simple Convolutional Auto Encoder, which has skip connections ( similar to a UNet ), whose input and output shapes are the same. We'll use `tf.keras.losses.mean_squared_error` and `tf.keras.optimizers.Adam`.


In [ ]:

alpha = 0.2

inputs = Input( shape=input_size )
conv1 = Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 )( inputs )
relu1 = LeakyReLU( alpha )( conv1 )
conv2 = Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 )( relu1 )
relu2 = LeakyReLU( alpha )( conv2 )
maxpool1 = MaxPooling2D()( relu2 )

conv3 = Conv2D( 64 , kernel_size=( 3 , 3 ) , strides=1 )( maxpool1 )
relu3 = LeakyReLU( alpha )( conv3 )
conv4 = Conv2D( 64 , kernel_size=( 3 , 3 ) , strides=1 )( relu3 )
relu4 = LeakyReLU( alpha )( conv4 )
maxpool2 = MaxPooling2D()( relu4 )

conv5 = Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 )( maxpool2 )
relu5 = LeakyReLU( alpha )( conv5 )
conv6 = Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 )( relu5 )
relu6 = LeakyReLU( alpha )( conv6 )
maxpool3 = MaxPooling2D()( relu6 )

conv7 = Conv2D( 256 , kernel_size=( 1 , 1 ) , strides=1 )( maxpool3 )
relu7 = LeakyReLU( alpha )( conv7 )
conv8 = Conv2D( 256 , kernel_size=( 1 , 1 ) , strides=1 )( relu7 )
relu8 = LeakyReLU( alpha )( conv8 )

upsample1 = UpSampling2D()( relu8 )
concat1 = Concatenate()([ upsample1 , conv6 ])
convtranspose1 = Conv2DTranspose( 128 , kernel_size=( 3 , 3 ) , strides=1)( concat1 )
relu9 = LeakyReLU( alpha )( convtranspose1 )
convtranspose2 = Conv2DTranspose( 128 , kernel_size=( 3 , 3 ) , strides=1  )( relu9 )
relu10 = LeakyReLU( alpha )( convtranspose2 )

upsample2 = UpSampling2D()( relu10 )
concat2 = Concatenate()([ upsample2 , conv4 ])
convtranspose3 = Conv2DTranspose( 64 , kernel_size=( 3 , 3 ) , strides=1)( concat2 )
relu11 = LeakyReLU( alpha )( convtranspose3 )
convtranspose4 = Conv2DTranspose( 64 , kernel_size=( 3 , 3 ) , strides=1 )( relu11 )
relu12 = LeakyReLU( alpha )( convtranspose4 )

upsample3 = UpSampling2D()( relu12 )
concat3 = Concatenate()([ upsample3 , conv2 ])
convtranspose5 = Conv2DTranspose( 32 , kernel_size=( 3 , 3 ) , strides=1)( concat3 )
relu13 = LeakyReLU( alpha )( convtranspose5 )
convtranspose6 = Conv2DTranspose( 3 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' )( relu13 )

model = tf.keras.models.Model( inputs , convtranspose6 )
model.compile( loss='mse' , optimizer='adam' , metrics=[ 'mse' ] )



## 4) Training the model

We'll train the model with a batch size of 50. To visualize the training process, you may use [TensorBoard](https://www.tensorflow.org/tensorboard). See [Using TensorBoard in Notebooks](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks).


In [ ]:

model.fit( x_train , y_train , epochs=30 , batch_size=25 , validation_data=( x_test , y_test ) )



## 5) Plotting the Results

We'll make some predictions on test images and plot the results. You'll probably see a square in the image where our model has constructed the image.


In [ ]:

fig = plt.figure(figsize=( 50 , 50 ))
for i in range( 1 , 6 ):
    pred = model.predict( x_test[ i : i+1 ] ) * 255
    fig.add_subplot( 1 , 10 , i )
    plt.imshow( pred[0].astype( np.uint8 )  )
plt.show()
